In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
from pathlib import Path

import torch

baseline_path = Path("/bigdata/cifar10/logs/rn18/baselines/1646668631/rn18_18_dNone_128_adam_l0.001_g0.1_w0.0_sTrue")
baseline_model = torch.load(baseline_path / "cnn_best.pth")
with open(baseline_path/'results.json') as json_file:
    baseline_result = json.load(json_file)
baseline_result

{'best_epoch': 229,
 'best_train_acc': 0.9999333333333333,
 'best_valid_acc': 0.9216,
 'best_valid_loss': 0.005079200401902199,
 'test_acc': 0.9137,
 'test_loss': 0.005285387974977494,
 'n_param': 11173962,
 'model_name': 'rn18_18_dNone_128_adam_l0.001_g0.1_w0.0_sTrue'}

In [3]:
baseline_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
import copy
from tddl.utils.model_stats import count_parameters
from tddl.factorizations import number_layers, listify_numbered_layers, get_weights
from tddl.dbs import find_error_given_c

baseline_count = count_parameters(baseline_model)
model = copy.deepcopy(baseline_model)
rn18_layers = [6,9,12,15,19,22,25,28,31,35,38,41,44,47,51,54,57,60,63]
rank = 0.5

numbered_layers = number_layers(model)
listed_layers = listify_numbered_layers(numbered_layers, layer_nrs=rn18_layers)
rank, c, error = find_error_given_c(listed_layers, desired_c = rank, baseline_count=baseline_count)

/home/jetzeschuurman/gitProjects/phd/tltorch/tltorch/factorized_tensors/core.py:145: UserWarning: Creating a subclass of FactorizedTensor TensorizedTensor with no name.
  warnings.warn(f'Creating a subclass of FactorizedTensor {cls.__name__} with no name.')
/home/jetzeschuurman/gitProjects/phd/tddl/venv/lib/python3.8/site-packages/tensorly/backend/core.py:885: UserWarning: In partial_svd: converting to NumPy. Check SVD_FUNS for available alternatives if you want to avoid this.
  warnings.warn('In partial_svd: converting to NumPy.'


0.5
factorize_layer.cache_info() = CacheInfo(hits=0, misses=5, maxsize=4096, currsize=5)
count_parameters.cache_info() = CacheInfo(hits=0, misses=2, maxsize=128, currsize=2)
factorize_layer.cache_info() = CacheInfo(hits=0, misses=10, maxsize=4096, currsize=10)
count_parameters.cache_info() = CacheInfo(hits=0, misses=3, maxsize=128, currsize=3)
factorize_layer.cache_info() = CacheInfo(hits=0, misses=16, maxsize=4096, currsize=16)
count_parameters.cache_info() = CacheInfo(hits=0, misses=4, maxsize=128, currsize=4)
factorize_layer.cache_info() = CacheInfo(hits=0, misses=21, maxsize=4096, currsize=21)
count_parameters.cache_info() = CacheInfo(hits=0, misses=5, maxsize=128, currsize=5)
factorize_layer.cache_info() = CacheInfo(hits=0, misses=26, maxsize=4096, currsize=26)
count_parameters.cache_info() = CacheInfo(hits=0, misses=6, maxsize=128, currsize=6)
factorize_layer.cache_info() = CacheInfo(hits=0, misses=31, maxsize=4096, currsize=31)
count_parameters.cache_info() = CacheInfo(hits=0, m

In [7]:
from tddl.factorizations import number_layers, listify_numbered_layers, get_weights

numbered_layers = number_layers(baseline_model)
layers = listify_numbered_layers(numbered_layers, layer_nrs=rn18_layers)

In [8]:
import numpy as np
from tddl.factorizations import factorize_layer

DESIRED_COMPRESSION = 0.5

rank = DESIRED_COMPRESSION
errors = np.array([])
for layer in layers:
    with torch.no_grad():
        fact_layer, error = factorize_layer(layer[2], 'tucker', rank, return_error=True)
    errors = np.append(errors, float(error.cpu().numpy()))

In [9]:
errors

array([0.36732996, 0.44806656, 0.45179987, 0.47657233, 0.44373474,
       0.51677138, 0.68679565, 0.53034019, 0.50432438, 0.46969157,
       0.45280796, 0.66759729, 0.44731942, 0.38146091, 0.37578163,
       0.29696083, 0.52024114, 0.20599779, 0.19827864])

In [10]:
error = errors.mean()
max_error = errors.max()
min_error = errors.min()

In [11]:
error

0.444309065216466

In [21]:
find_error_given_c(layers, desired_c = DESIRED_COMPRESSION, error=error, max_error=max_error, min_error=min_error, baseline_count=baseline_count)

0.444309065216466
factorize_layer.cache_info() = CacheInfo(hits=0, misses=6, maxsize=4096, currsize=6)
count_parameters.cache_info() = CacheInfo(hits=0, misses=1, maxsize=128, currsize=1)
factorize_layer.cache_info() = CacheInfo(hits=0, misses=7, maxsize=4096, currsize=7)
count_parameters.cache_info() = CacheInfo(hits=0, misses=2, maxsize=128, currsize=2)
factorize_layer.cache_info() = CacheInfo(hits=0, misses=8, maxsize=4096, currsize=8)
count_parameters.cache_info() = CacheInfo(hits=0, misses=3, maxsize=128, currsize=3)
factorize_layer.cache_info() = CacheInfo(hits=0, misses=12, maxsize=4096, currsize=12)
count_parameters.cache_info() = CacheInfo(hits=0, misses=4, maxsize=128, currsize=4)
factorize_layer.cache_info() = CacheInfo(hits=0, misses=13, maxsize=4096, currsize=13)
count_parameters.cache_info() = CacheInfo(hits=0, misses=5, maxsize=128, currsize=5)
factorize_layer.cache_info() = CacheInfo(hits=0, misses=16, maxsize=4096, currsize=16)
count_parameters.cache_info() = CacheInfo

([0.703125,
  0.84375,
  0.84375,
  0.875,
  0.84375,
  0.921875,
  1.0,
  0.9375,
  0.90625,
  0.90625,
  0.84375,
  1.0,
  0.84375,
  0.75,
  0.75,
  0.5625,
  1.0,
  0.21875,
  0.1875],
 0.5038428625406101,
 tensor(0.2777, device='cuda:0'))

In [20]:
import gc
import functools

gc.collect()

# All objects collected
objects = [
    i for i in gc.get_objects()
    if isinstance(i, functools._lru_cache_wrapper)
]

# All objects cleared
for object in objects:
    object.cache_clear()

# factorize_layer.cache_info()
# count_parameters.cache_info()
# find_rank_given_error.cache_info()

In [19]:
find_error_given_c(layers, desired_c = DESIRED_COMPRESSION, baseline_count=baseline_count)

0.5


/home/jetzeschuurman/gitProjects/phd/tddl/venv/lib/python3.8/site-packages/tensorly/backend/core.py:885: UserWarning: In partial_svd: converting to NumPy. Check SVD_FUNS for available alternatives if you want to avoid this.
  warnings.warn('In partial_svd: converting to NumPy.'


factorize_layer.cache_info() = CacheInfo(hits=0, misses=5, maxsize=4096, currsize=5)
count_parameters.cache_info() = CacheInfo(hits=0, misses=1, maxsize=128, currsize=1)
factorize_layer.cache_info() = CacheInfo(hits=0, misses=10, maxsize=4096, currsize=10)
count_parameters.cache_info() = CacheInfo(hits=0, misses=2, maxsize=128, currsize=2)
factorize_layer.cache_info() = CacheInfo(hits=0, misses=16, maxsize=4096, currsize=16)
count_parameters.cache_info() = CacheInfo(hits=0, misses=3, maxsize=128, currsize=3)
factorize_layer.cache_info() = CacheInfo(hits=0, misses=21, maxsize=4096, currsize=21)
count_parameters.cache_info() = CacheInfo(hits=0, misses=4, maxsize=128, currsize=4)
factorize_layer.cache_info() = CacheInfo(hits=0, misses=26, maxsize=4096, currsize=26)
count_parameters.cache_info() = CacheInfo(hits=0, misses=5, maxsize=128, currsize=5)
factorize_layer.cache_info() = CacheInfo(hits=0, misses=31, maxsize=4096, currsize=31)
count_parameters.cache_info() = CacheInfo(hits=0, misse

([0.6875,
  0.84375,
  0.84375,
  0.875,
  0.84375,
  0.90625,
  1.0,
  0.9375,
  0.90625,
  0.875,
  0.84375,
  1.0,
  0.84375,
  0.75,
  0.75,
  0.5625,
  1.0,
  0.1875,
  0.1875],
 0.4962715999929121,
 tensor(0.2777, device='cuda:0'))